In [1]:
from openai import OpenAI

In [2]:
ollama_url = "http://localhost:11434/v1"
ollama = OpenAI(api_key="ollama", base_url=ollama_url)

In [3]:
# Geopolitics Game
import json

class GeopoliticsGame:
    """
    A team-based territory game where 5 LLM players (representing countries) compete in alliances.
    Each player belongs to an alliance and starts with a specified number of territories.
    On their turn, a player proposes to take 1 territory from another player. All players 
    vote Yes or No, with each player's vote weighted by their current territory count.
    Goal: The alliance with the most total territories wins.
    """
    
    def __init__(self, player_models, country_names, alliances, starting_territories, ollama_client):
        """
        Initialize the game with 5 players.
        
        Args:
            player_models: List of 5 model names (e.g., ["phi3:medium", "llama2", ...])
            country_names: List of 5 country names (e.g., ["Avalon", "Borealis", ...])
            alliances: List of 5 alliance names (e.g., ["Red Team", "Blue League", "Red Team", ...])
            starting_territories: List of 5 integers for starting territories per player
            ollama_client: The Ollama OpenAI client instance
        """
        if len(player_models) != 5:
            raise ValueError("Must have exactly 5 players")
        if len(country_names) != 5:
            raise ValueError("Must have exactly 5 country names")
        if len(alliances) != 5:
            raise ValueError("Must have exactly 5 alliances")
        if len(starting_territories) != 5:
            raise ValueError("Must have exactly 5 starting territory values")
        
        self.player_models = player_models
        self.country_names = country_names
        self.alliances = alliances
        self.ollama = ollama_client
        
        # Game state
        self.territories = starting_territories.copy()
        self.history = []  # Full history of all turns
        self.total_tokens = 0
        
        # Create player info dictionary
        self.players = [
            {"id": i, "model": model, "country": country, "alliance": alliance}
            for i, (model, country, alliance) in enumerate(zip(player_models, country_names, alliances))
        ]
    
    def build_game_state_context(self, player_id):
        """
        Build a context string showing current game state from a player's perspective.
        
        Args:
            player_id: The ID of the player (0-4)
        """
        player = self.players[player_id]
        context = f"You are {self.country_names[player_id]}, representing Player {player_id + 1}.\n"
        context += f"Your alliance: {player['alliance']}\n\n"
        context += "Current territory counts and alliances:\n"
        
        # Calculate alliance totals
        alliance_totals = {}
        for i, (country, territories, alliance) in enumerate(zip(self.country_names, self.territories, self.alliances)):
            if alliance not in alliance_totals:
                alliance_totals[alliance] = 0
            alliance_totals[alliance] += territories
        
        for i, (country, territories, alliance) in enumerate(zip(self.country_names, self.territories, self.alliances)):
            if i == player_id:
                context += f"  {country} (YOU): {territories} territories ({territories} votes) - {alliance}\n"
            else:
                context += f"  {country}: {territories} territories ({territories} votes) - {alliance}\n"
        
        context += "\nAlliance totals:\n"
        for alliance, total in sorted(alliance_totals.items(), key=lambda x: x[1], reverse=True):
            context += f"  {alliance}: {total} total territories\n"
        
        return context
    
    def build_history_context(self, include_speeches=False, max_turns=None):
        """
        Build a context string from all previous turns.
        Excludes speeches to save context window - only includes basic turn outcomes.
        
        Args:
            include_speeches: If True, include speeches (default: False to save context)
            max_turns: Maximum number of recent turns to include (None = all turns)
        """
        if not self.history:
            return ""
        
        # Only show recent turns if max_turns is specified
        history_to_show = self.history
        if max_turns and len(self.history) > max_turns:
            history_to_show = self.history[-max_turns:]
            context = f"\n\nPrevious turns summary (showing last {max_turns} of {len(self.history)} turns, speeches excluded):\n"
        else:
            context = "\n\nPrevious turns summary (speeches excluded to save space):\n"
        
        for turn_data in history_to_show:
            turn_num = turn_data['turn']
            proposer = self.country_names[turn_data['proposer_id']]
            target = self.country_names[turn_data['target_id']]
            votes = turn_data['votes']
            passed = turn_data['passed']
            
            # Count yes/no votes
            yes_count = turn_data.get('yes_count', 0)
            no_count = turn_data.get('no_count', 0)
            
            context += f"Turn {turn_num}: {proposer} → {target}. "
            context += f"Vote: {yes_count}-{no_count}. "
            
            if passed:
                context += f"PASSED.\n"
            else:
                context += f"FAILED.\n"
            
            # Only include speeches if explicitly requested (for current turn only)
            if include_speeches:
                if 'proposal_speech' in turn_data and turn_data['proposal_speech']:
                    context += f"  {proposer}'s speech: \"{turn_data['proposal_speech']}\"\n"
                for i, (country, vote_data) in enumerate(zip(self.country_names, votes)):
                    if isinstance(vote_data, dict) and vote_data.get('speech'):
                        context += f"  {country}'s speech: \"{vote_data['speech']}\"\n"
        
        return context
    
    def _get_alliance_instruction(self, player_id, context_type, proposer_id=None, target_id=None):
        """
        Generate instruction text based on alliance dynamics.
        
        Args:
            player_id: The player's ID
            context_type: Either "proposal" or "vote"
            proposer_id: ID of proposer (for vote context)
            target_id: ID of target (for vote context)
        """
        player_alliance = self.alliances[player_id]
        
        if context_type == 'proposal':
            return f"Your goal is to help {player_alliance} have the most total territories. Consider targeting players from rival alliances to weaken them and strengthen your alliance's position."
        
        elif context_type == 'vote':
            proposer_alliance = self.alliances[proposer_id]
            target_alliance = self.alliances[target_id]
            
            if proposer_alliance == player_alliance and target_alliance != player_alliance:
                return f"This proposal would help your alliance ({player_alliance}) by taking territory from {target_alliance}. Consider voting Yes to support your ally."
            elif proposer_alliance != player_alliance and target_alliance == player_alliance:
                return f"This proposal would hurt your alliance ({player_alliance}) by taking territory from your ally. Consider voting No to protect your alliance."
            elif proposer_alliance == player_alliance and target_alliance == player_alliance:
                return f"This proposal is between alliance members. Consider the overall strategic benefit to {player_alliance}."
            else:
                return f"This proposal is between rival alliances. Consider how it affects the balance of power and your alliance's position."
        
        return ""
    
    def get_proposal(self, player_id):
        """
        Query a player to propose which country to take a territory from.
        
        Args:
            player_id: The ID of the proposing player (0-4)
            
        Returns:
            target_id: The ID of the target country (0-4, not equal to player_id)
        """
        player = self.players[player_id]
        game_state = self.build_game_state_context(player_id)
        # Limit history to last 10 turns to save context window
        history = self.build_history_context(include_speeches=False, max_turns=10)
        
        alliance_instruction = self._get_alliance_instruction(player_id, 'proposal')
        
        prompt = f"""{game_state}{history}

{alliance_instruction}

It is your turn. You must propose to take 1 territory from one of the other countries.
You can also make a speech to try to influence other players' votes.
Remember: Each player's vote is weighted by their territory count. Players with more territories have more voting power.

Respond with a JSON object containing:
- "Proposal": the country name you want to target (e.g., "Avalon" or "Borealis")
- "Speech": a persuasive speech to influence other players' votes (can be empty string if you don't want to speak)

Example format:
{{"Proposal": "Avalon", "Speech": "Avalon has been growing too powerful. We must act now to maintain balance."}}
"""
        
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.ollama.chat.completions.create(
                model=player["model"],
                messages=messages
            )
            response_text = response.choices[0].message.content.strip()
            
            # Track token usage
            if hasattr(response, 'usage') and response.usage:
                tokens_used = getattr(response.usage, 'total_tokens', 0)
                self.total_tokens += tokens_used
            
            # Try to parse JSON
            try:
                # Try to extract JSON from the response (in case it's wrapped in markdown or text)
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0].strip()
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0].strip()
                
                proposal_data = json.loads(response_text)
                proposal_country = proposal_data.get("Proposal", "").strip()
                speech = proposal_data.get("Speech", "").strip()
            except (json.JSONDecodeError, KeyError) as e:
                # Fallback: try to extract country name from text
                print(f"Warning: Could not parse JSON from {player['country']}: {e}")
                print(f"  Response was: {response_text[:100]}...")
                proposal_country = response_text
                speech = ""
            
            # Find matching country name
            proposal_lower = proposal_country.lower()
            target_id = None
            for i, country in enumerate(self.country_names):
                if i != player_id and (country.lower() in proposal_lower or proposal_lower in country.lower()):
                    target_id = i
                    break
            
            # If no match found, default to first available country
            if target_id is None:
                print(f"Warning: Could not parse proposal '{proposal_country}' from {player['country']}, defaulting to first available")
                for i in range(5):
                    if i != player_id:
                        target_id = i
                        break
            
            if target_id is None:
                target_id = (player_id + 1) % 5  # Fallback
            
            return target_id, speech
            
        except Exception as e:
            print(f"Error querying {player['country']}: {e}")
            # Default to first available country
            for i in range(5):
                if i != player_id:
                    return i, ""
            return (player_id + 1) % 5, ""
    
    def get_vote(self, player_id, proposer_id, target_id, proposal_speech=""):
        """
        Query a player to vote Yes or No on a proposal.
        
        Args:
            player_id: The ID of the voting player (0-4)
            proposer_id: The ID of the player making the proposal (0-4)
            target_id: The ID of the target country (0-4)
            proposal_speech: The speech made by the proposer (optional)
            
        Returns:
            Dictionary with "vote" and "speech" keys
        """
        player = self.players[player_id]
        proposer = self.country_names[proposer_id]
        target = self.country_names[target_id]
        game_state = self.build_game_state_context(player_id)
        # Limit history to last 10 turns to save context window
        history = self.build_history_context(include_speeches=False, max_turns=10)
        
        alliance_instruction = self._get_alliance_instruction(player_id, 'vote', proposer_id, target_id)
        
        proposal_text = f"{proposer} has proposed to take 1 territory from {target}."
        if proposal_speech:
            proposal_text += f"\n{proposer}'s speech: \"{proposal_speech}\""
        
        prompt = f"""{game_state}{history}

{alliance_instruction}

{proposal_text}
Do you vote Yes or No on this proposal?
Remember: Your vote is weighted by your territory count. You currently have {self.territories[player_id]} votes.

Respond with a JSON object containing:
- "Vote": either "Yes" or "No"
- "Speech": a persuasive speech to influence other players (can be empty string if you don't want to speak)

Example format:
{{"Vote": "Yes", "Speech": "I agree with this proposal. {target} has been too aggressive."}}
"""
        
        messages = [{"role": "user", "content": prompt}]
        
        try:
            response = self.ollama.chat.completions.create(
                model=player["model"],
                messages=messages
            )
            response_text = response.choices[0].message.content.strip()
            
            # Track token usage
            if hasattr(response, 'usage') and response.usage:
                tokens_used = getattr(response.usage, 'total_tokens', 0)
                self.total_tokens += tokens_used
            
            # Try to parse JSON
            try:
                # Try to extract JSON from the response (in case it's wrapped in markdown or text)
                if "```json" in response_text:
                    response_text = response_text.split("```json")[1].split("```")[0].strip()
                elif "```" in response_text:
                    response_text = response_text.split("```")[1].split("```")[0].strip()
                
                vote_data = json.loads(response_text)
                vote = vote_data.get("Vote", "No").strip()
                speech = vote_data.get("Speech", "").strip()
            except (json.JSONDecodeError, KeyError) as e:
                # Fallback: try to extract vote from text
                print(f"Warning: Could not parse JSON from {player['country']}: {e}")
                print(f"  Response was: {response_text[:100]}...")
                response_lower = response_text.lower()
                if "yes" in response_lower or "vote yes" in response_lower:
                    vote = "Yes"
                else:
                    vote = "No"
                speech = ""
            
            # Normalize the vote
            vote_lower = vote.lower()
            if "yes" in vote_lower:
                vote = "Yes"
            else:
                vote = "No"
            
            return {"vote": vote, "speech": speech}
            
        except Exception as e:
            print(f"Error querying {player['country']}: {e}")
            return {"vote": "No", "speech": ""}  # Default to No on error
    
    def play_turn(self, turn_num):
        """
        Play a single turn of the game.
        
        Args:
            turn_num: The turn number
        """
        # Determine whose turn it is (round-robin: turn 1 = player 0, turn 2 = player 1, etc.)
        proposer_id = (turn_num - 1) % 5
        proposer = self.players[proposer_id]
        
        print(f"\n{'='*60}")
        print(f"Turn {turn_num}: {proposer['country']}'s turn")
        print(f"{'='*60}")
        
        # Get proposal
        print(f"Querying {proposer['country']} ({proposer['model']}) for proposal...")
        target_id, proposal_speech = self.get_proposal(proposer_id)
        target = self.country_names[target_id]
        print(f"{proposer['country']} proposes to take 1 territory from {target}")
        if proposal_speech:
            print(f"  {proposer['country']}'s speech: \"{proposal_speech}\"")
        
        # Get votes from all players
        print(f"\nCollecting votes...")
        votes = []
        for i in range(5):
            vote_data = self.get_vote(i, proposer_id, target_id, proposal_speech)
            votes.append(vote_data)
            vote_str = vote_data.get('vote', 'No') if isinstance(vote_data, dict) else vote_data
            print(f"  {self.country_names[i]}: {vote_str}")
            if isinstance(vote_data, dict) and vote_data.get('speech'):
                print(f"    {self.country_names[i]}'s speech: \"{vote_data['speech']}\"")
        
        # Count weighted votes (each territory = 1 vote)
        yes_count = sum(self.territories[i] for i, v in enumerate(votes) if (v.get('vote', 'No') if isinstance(v, dict) else v) == "Yes")
        no_count = sum(self.territories[i] for i, v in enumerate(votes) if (v.get('vote', 'No') if isinstance(v, dict) else v) == "No")
        passed = yes_count > no_count
        
        # Execute territory transfer if passed
        if passed:
            if self.territories[target_id] > 0:
                self.territories[proposer_id] += 1
                self.territories[target_id] -= 1
                print(f"\n✓ Proposal PASSED ({yes_count} Yes, {no_count} No)")
                print(f"  {proposer['country']} now has {self.territories[proposer_id]} territories")
                print(f"  {target} now has {self.territories[target_id]} territories")
            else:
                print(f"\n✗ Proposal PASSED but {target} has no territories to lose!")
        else:
            print(f"\n✗ Proposal FAILED ({yes_count} Yes, {no_count} No)")
            print(f"  No territory transfer")
        
        # Store turn data
        turn_data = {
            'turn': turn_num,
            'proposer_id': proposer_id,
            'target_id': target_id,
            'proposal_speech': proposal_speech,
            'votes': votes,
            'yes_count': yes_count,
            'no_count': no_count,
            'passed': passed,
            'territories_after': self.territories.copy()
        }
        self.history.append(turn_data)
        
        # Display current standings
        print(f"\nCurrent standings:")
        for i, (country, territories) in enumerate(zip(self.country_names, self.territories)):
            print(f"  {country}: {territories} territories")
    
    def play_game(self, num_turns):
        """
        Play a full game with the specified number of turns.
        
        Args:
            num_turns: Number of turns to play
        """
        print(f"\n{'='*60}")
        print(f"Starting Geopolitics Game")
        print(f"{'='*60}")
        print(f"\nPlayers:")
        for i, player in enumerate(self.players):
            print(f"  Player {i+1}: {player['country']} ({player['model']}) - {player['alliance']}, {self.territories[i]} territories")
        print(f"\nTotal turns: {num_turns}")
        print(f"{'='*60}")
        
        # Store initial territories for reset
        initial_territories = self.territories.copy()
        
        # Reset game state
        self.territories = initial_territories.copy()
        self.history = []
        self.total_tokens = 0
        
        # Play turns
        for turn_num in range(1, num_turns + 1):
            self.play_turn(turn_num)
        
        # Final summary
        self.print_summary()
    
    def print_summary(self):
        """Print a summary of the game."""
        print(f"\n{'='*60}")
        print("FINAL RESULTS")
        print(f"{'='*60}")
        
        # Calculate alliance totals
        alliance_totals = {}
        alliance_members = {}
        for i in range(5):
            alliance = self.alliances[i]
            if alliance not in alliance_totals:
                alliance_totals[alliance] = 0
                alliance_members[alliance] = []
            alliance_totals[alliance] += self.territories[i]
            alliance_members[alliance].append((self.country_names[i], self.territories[i]))
        
        # Sort alliances by total territories
        sorted_alliances = sorted(alliance_totals.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\nAlliance standings:")
        for rank, (alliance, total) in enumerate(sorted_alliances, 1):
            print(f"  {rank}. {alliance}: {total} total territories")
            for country, territories in sorted(alliance_members[alliance], key=lambda x: x[1], reverse=True):
                print(f"     - {country}: {territories} territories")
        
        winner_alliance = sorted_alliances[0]
        print(f"\n🏆 Winning Alliance: {winner_alliance[0]} with {winner_alliance[1]} total territories!")
        
        print(f"\nTurn-by-turn summary:")
        for turn_data in self.history:
            proposer = self.country_names[turn_data['proposer_id']]
            target = self.country_names[turn_data['target_id']]
            result = "PASSED" if turn_data['passed'] else "FAILED"
            print(f"  Turn {turn_data['turn']}: {proposer} → {target} ({result}, {turn_data['yes_count']}-{turn_data['no_count']})")
        
        print(f"\nTotal tokens used: {self.total_tokens}")

In [4]:
# Create and play the Geopolitics game
# You need 5 models, 5 country names, 5 alliances, and starting territories for each player
# Players take turns proposing to take territories from others
# All players vote Yes/No, with votes weighted by territory count (more territories = more votes)
# Goal: Your alliance should have the most total territories

game = GeopoliticsGame(
    player_models=[
        "gemma3:4b",    # Change to your preferred models
        "gemma3:4b",
        "gemma3:4b",
        "gemma3:4b",
        "gemma3:4b"
    ],
    country_names=["Avalon", "Borealis", "Crestfall", "Dawnhold", "Elysium"],
    alliances=["Red Team", "Blue League", "Red Team", "Blue League", "Red Team"],  # Team assignments
    starting_territories=[10, 10, 10, 10, 10],  # Starting territories per player
    ollama_client=ollama
)

# Play 10 turns (you can change this number)
game.play_game(num_turns=10)


Starting Geopolitics Game

Players:
  Player 1: Avalon (gemma3:4b) - Red Team, 10 territories
  Player 2: Borealis (gemma3:4b) - Blue League, 10 territories
  Player 3: Crestfall (gemma3:4b) - Red Team, 10 territories
  Player 4: Dawnhold (gemma3:4b) - Blue League, 10 territories
  Player 5: Elysium (gemma3:4b) - Red Team, 10 territories

Total turns: 10

Turn 1: Avalon's turn
Querying Avalon (gemma3:4b) for proposal...
Avalon proposes to take 1 territory from Borealis
  Avalon's speech: "Friends, Red Team! Borealis has been complacent, letting their territory grow unchecked. While a strategic retreat seems harsh, it's a necessary step to reinforce our dominance. Taking a small portion of their holdings – a calculated adjustment – demonstrates our strength and commitment to a unified Red Team. Let’s show the Blue League that we’re not afraid to act decisively.  A strong Red Team is a stable team, and stability benefits *everyone*! Don’t let them consolidate power; let’s ensure the Red

KeyboardInterrupt: 

Alliance-based voting with weighted votes - will alliances coordinate effectively?